# Clasificacion supervisada Support Vector Machine

In [1]:
# Importar paquete GEE
import ee
ee.Initialize()

## Filtro zona estudio mediante GEE

In [2]:
# Agregar un shp GEE
zona_ee = ee.FeatureCollection("users/bravomoralesnino/SHP/ZONA_19S")

In [3]:
# Importar geemap
import geemap
Map = geemap.Map(basemap='ESRI')

In [4]:
geometria = zona_ee.geometry()
Map.centerObject(geometria, 11)

In [5]:
Map.addLayer(zona_ee, {"color" : "red"},name = "Zona estudio")
Map

Map(center=[-12.908744902436744, -69.9947155021359], controls=(WidgetControl(options=['position', 'transparent…

## Seleccion de imagen

In [6]:
# Seleccionar una imagen Landsat8
img_L8 = ee.Image("LANDSAT/LC08/C01/T1_SR/LC08_003069_20190808")\
             .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7'],
                     ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2'])\
             .multiply(0.0001)\
             .clip(zona_ee)\
             .reproject(crs="EPSG:32719", scale = 30)

In [7]:
# Nombre de las bandas
img_L8.bandNames().getInfo()

['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2']

In [8]:
# Simbologia escala 0 - 1
viz_es = {
    'bands': ['SWIR1','NIR','RED'],
    'min': 0.1,
    'max': 0.6,
    'gamma': 1.6
}

In [9]:
Map.addLayer(img_L8, viz_es,name = "Landsat8 RS")
Map

Map(bottom=140945.0, center=[-13.032655603554861, -69.67945573844864], controls=(WidgetControl(options=['posit…

## Agregar ROI estudio SHP

In [10]:
import os
os.getcwd() # Consultar ruta trabajo

'C:\\Users\\GEOMATICA\\Teledeteccion_Python\\GEE'

In [11]:
ruta_archivos = r"E:\Teledeteccion_GEE\Python\Datos_Espaciales\SHP\ROI"
os.chdir(ruta_archivos) # Cambiar ruta trabajo
os.getcwd() # Consultar ruta trabajo

'E:\\Teledeteccion_GEE\\Python\\Datos_Espaciales\\SHP\\ROI'

In [12]:
import geopandas as gpd

In [13]:
# cargar un shp
ROI = gpd.read_file("ROI_L8_Geo.shp")
ROI

,Id,Class,ID_Clas,geometry
0,1,Bosque,3,POINT (-69.95182 -12.96576)
1,2,Bosque,3,POINT (-69.95296 -12.94446)
2,3,Bosque,3,POINT (-69.95096 -12.93107)
3,4,Bosque,3,POINT (-69.90221 -12.93842)
4,5,Bosque,3,POINT (-69.88706 -12.93034)
...,...,...,...,...
195,197,Agricultura,1,POINT (-70.02166 -12.99661)
196,198,Agricultura,1,POINT (-70.01474 -12.99581)
197,199,Agricultura,1,POINT (-70.01503 -12.86148)
198,200,Agricultura,1,POINT (-70.00675 -12.85307)


In [14]:
# Crear un featureCollecion mediante GeoJSON
ee_ROI = ee.FeatureCollection(ROI.__geo_interface__)

In [15]:
Map.addLayer(ee_ROI,{"color" : "red"}, name = "L8 ROI")
Map

Map(bottom=281425.0, center=[-12.922252875636763, -69.95407104492189], controls=(WidgetControl(options=['posit…

## Datos de entrenamiento ROI

In [16]:
# Obtener un FeatureCollection de datos de entrenamiento
training = img_L8.sampleRegions(collection = ee_ROI,
                                properties = ["ID_Clas","Class"],
                                scale = 30)

In [17]:
# seleccionar las bandas
bandas = ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2']

## Parámetro clasificación SVM

In [18]:
# image classification
classifier = ee.Classifier.libsvm().train(training,"ID_Clas",bandas)

## Aplicar clasificación

In [19]:
# Clasificador 
class_SVM = img_L8.classify(classifier)

In [20]:
# Simbologia clasificacion
viz_clas = {
    'palette': ["#4dde40","#2032ed",
                "#13800c","#dc0f0f"],
    'min': 1,
    'max': 4
}

In [39]:
Map.addLayer(class_SVM, viz_clas,name = "Clasificado SVM")
Map

Map(bottom=281521.0, center=[-12.986493155428498, -69.9437713623047], controls=(WidgetControl(options=['positi…

In [22]:
class_SVM.bandNames().getInfo()

['classification']

## Matriz de confusión y precisión

In [23]:
# Estimar la matriz de confusión o error 
Matriz = classifier.confusionMatrix().getInfo()
Matriz

[[0, 0, 0, 0, 0],
 [0, 2, 0, 30, 0],
 [0, 0, 45, 0, 0],
 [0, 0, 0, 49, 0],
 [0, 0, 0, 0, 74]]

In [24]:
# precisión de la clasificación
classifier.confusionMatrix().accuracy().getInfo()

0.85

## Exportar Imagen 

In [25]:
os.getcwd()

'E:\\Teledeteccion_GEE\\Python\\Datos_Espaciales\\SHP\\ROI'

In [40]:
ruta_archivos = r"E:\Teledeteccion_GEE\Python\Datos_Espaciales\Resultado_GEE"
os.chdir(ruta_archivos) # Cambiar ruta trabajo
os.getcwd() # Consultar ruta trabajo

'E:\\Teledeteccion_GEE\\Python\\Datos_Espaciales\\Resultado_GEE'

In [41]:
# Exportar imagen Clasificado
geemap.ee_export_image(class_SVM, filename = "L8_class_SVM2.tif", scale=30, region=geometria, file_per_band=False)

Generating URL ...
Please wait ...
An error occurred while downloading.


In [42]:
geemap.ee_export_image_to_drive(class_SVM, description='L8_class_SVM', folder='GEE_Python', region=geometria, scale=30)

Exporting L8_class_SVM ...
